In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
import sys
sys.path.append('../')
from gis_resources import san_diego_county_zips
import os
from arcgis.geocoding import geocode
from arcgis.features import GeoAccessor

In [2]:

#gis = GIS(username='izaslavsky_ucsd')  # this will ask for password. 
print("Enter the Client ID/APP ID generated from Arc GIS:")
app_id = input()
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)
arcgis.__version__
# Elm5V3upnnV17Q3r

Enter the Client ID/APP ID generated from Arc GIS:


 Elm5V3upnnV17Q3r


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=ywMFwANSpjPDDsIt9Md7yIh0Dc7qhm&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'2.1.0'

In [ ]:
print("Nourish User:")
nourish_user = input()
print("Nourish Password:")
nourish_pswd = input()

In [4]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [5]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from ca_farms_2')

        # display the PostgreSQL database server version
result = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [6]:
ca_farms_result_list = [(row[1],row[2],row[5],row[6],row[7]) for row in result]

In [7]:
# Create a DataFrame
dataFrame = pd.DataFrame(ca_farms_result_list,
              columns=("company_name","farm_address", "crops_product","sells_at", "practices")
              );
dataFrame

,company_name,farm_address,crops_product,sells_at,practices
0,SAND N STRAW,"629 Mar Vista Dr, Vista, California, 92081",[An abundance of naturally grown local fruit a...,[CSA],"[No information about practices, services & a..."
1,"FOUR SEASONS MICROGREENS, LLC","111 Marvis Dr., Atwater, California, 95301","[Microgreens: salad mix, arugula, peas, sun...","[CSA, Farmers Markets, Online, Restaurants]","[Beginning farmers, Woman-owned, Farmer of C..."
2,GUZMAN AND SON’S PRODUCE,"5241 Arena Way, Atwater, California, 95301","[Jicama, Sweet potatoes, cherry tomatoes, b...","[CSA, Farmers Markets, Online, Wholesale]","[Certified Organic, Woman-owned, Farmer of C..."
3,FENTON FAMILY FARM,"2235 Riverview Rd, Fallbrook, California, 92028",[everything from Apples to Zapotes],"[Farmers Markets, Farm Stand, Wholesale, Re...","[Farming organically (non-certified), Permacu..."
4,THREE FEATHERS FARM,"14490 Watsonville Rd., Morgan Hill, California...",[Speciality Crop],"[Farmers Markets, Wholesale, Restaurants]","[Beginning farmers, Farmer of Color-owned, H..."
...,...,...,...,...,...
633,MACEDO’S MINI ACRE,"11175 Golf Link Rd, Turlock, California, 95380","[alpaca fleeces, dryer balls, yarn, felt, ...","[Farm Stand, Online, Wholesale]","[Farming organically (non-certified), Pasture..."
634,WILD OAT HOLLOW,"Penngrove, California, 94951","[Yarn, roving, raw wool, sheep skin, clima...","[Farm Stand, Online]","[Farming organically (non-certified), No-Till..."
635,BARBELL FARM,"10466 Franklin Blvd, Elk Grove, California, 95757","[Pastured Poultry, Eggs, Pork, vegetables.]","[CSA, Farm Stand, Online]","[Farming organically (non-certified), Climate..."
636,PAUL’S PRODUCE,"19655 Arnold Dr, Sonoma, California, 95476","[Fruits, vegetables]",[Wholesale],"[No information about practices, services & a..."


In [8]:
# Trying to capture the zip from farm address using regex
dataFrame['zip'] = dataFrame['farm_address'].str.extract('^.*, ([0-9]{5})$', expand=True)

In [9]:
# Just in case we don't have a farm address
# dataFrame = dataFrame.astype({"zip": int})
# dataFrame = dataFrame[~dataFrame['zip'].isna()]

In [10]:
# Pick the data for only San Diego County
san_diego_county_zips_lst = san_diego_county_zips()

In [11]:
san_diego_county_zips_lst[:3]

['91910', '91911', '91913']

In [12]:
dataFrame = dataFrame[dataFrame["zip"].isin(san_diego_county_zips())]
dataFrame.shape

(48, 6)

In [13]:
dataFrame['crops_product']=dataFrame.crops_product.apply(lambda x: ','.join(x))
dataFrame['sells_at']=dataFrame.sells_at.apply(lambda x: ','.join(x))
dataFrame['practices']=dataFrame.practices.apply(lambda x: ','.join(x))

/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_60699/2305921363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['crops_product']=dataFrame.crops_product.apply(lambda x: ','.join(x))
/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_60699/2305921363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame['sells_at']=dataFrame.sells_at.apply(lambda x: ','.join(x))
/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_60699/2305921363.py:3: SettingWithCopyWarning: 

In [14]:
def get_lat_long(address):
    loc = geocode(address)[0]['location']
    return loc['x'],loc['y']

In [15]:
dataFrame[['longitude','latitude']] = dataFrame.apply(lambda row: get_lat_long(row.farm_address), axis='columns', result_type='expand')
print("Shape:",dataFrame.shape)
dataFrame.head(5)

Shape: (48, 8)


/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_60699/1041540729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[['longitude','latitude']] = dataFrame.apply(lambda row: get_lat_long(row.farm_address), axis='columns', result_type='expand')
/var/folders/jg/d8_zwblj5ql38q843xmy7h9r0000gp/T/ipykernel_60699/1041540729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[['longitude','latitude']] = dataFrame.apply(lambda row: get_lat_long(row.farm_address), axis='columns', result

,company_name,farm_address,crops_product,sells_at,practices,zip,longitude,latitude
0,SAND N STRAW,"629 Mar Vista Dr, Vista, California, 92081",An abundance of naturally grown local fruit an...,CSA,"No information about practices, services & att...",92081,-117.235804,33.181435
3,FENTON FAMILY FARM,"2235 Riverview Rd, Fallbrook, California, 92028",everything from Apples to Zapotes,"Farmers Markets, Farm Stand, Wholesale, Restau...","Farming organically (non-certified), Permacult...",92028,-117.309912,33.422165
9,RANDOL RIDGE RANCH,"Ramona, California, 92065",Vegetables and fruits,"CSA, Farm Stand",Farming organically (non-certified),92065,-116.866480,33.042215
15,HILLCREST HEALTHY GROVE,"744 Hillcrest Ln, Fallbrook, California, 92028",Avocadoes (available late January through May)...,Online,Farming organically (non-certified),92028,-117.243262,33.391117
55,GARDEN OF EDEN ORGANICS,"9928 Protea Gardens Rd, Escondido, California,...","Strawberries, Peaches, Apples, Radishes, Cucum...",CSA,Certified Organic,92026,-117.119996,33.195886


In [16]:
gdf = geopandas.GeoDataFrame(dataFrame, geometry=geopandas.points_from_xy(dataFrame.longitude, dataFrame.latitude))

In [17]:
ca_farms_sgdf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

In [18]:
#ca_farms_sgdf['zip'] = ca_farms_sgdf['zip'].astype(str)

In [19]:
ca_farms_sgdf.head(5)

,company_name,farm_address,crops_product,sells_at,practices,zip,longitude,latitude,SHAPE
0,SAND N STRAW,"629 Mar Vista Dr, Vista, California, 92081",An abundance of naturally grown local fruit an...,CSA,"No information about practices, services & att...",92081,-117.235804,33.181435,"{""x"": -117.23580367734463, ""y"": 33.18143519721..."
3,FENTON FAMILY FARM,"2235 Riverview Rd, Fallbrook, California, 92028",everything from Apples to Zapotes,"Farmers Markets, Farm Stand, Wholesale, Restau...","Farming organically (non-certified), Permacult...",92028,-117.309912,33.422165,"{""x"": -117.30991168753704, ""y"": 33.42216454605..."
9,RANDOL RIDGE RANCH,"Ramona, California, 92065",Vegetables and fruits,"CSA, Farm Stand",Farming organically (non-certified),92065,-116.866480,33.042215,"{""x"": -116.86647999999997, ""y"": 33.04221500000..."
15,HILLCREST HEALTHY GROVE,"744 Hillcrest Ln, Fallbrook, California, 92028",Avocadoes (available late January through May)...,Online,Farming organically (non-certified),92028,-117.243262,33.391117,"{""x"": -117.24326197860914, ""y"": 33.39111747378..."
55,GARDEN OF EDEN ORGANICS,"9928 Protea Gardens Rd, Escondido, California,...","Strawberries, Peaches, Apples, Radishes, Cucum...",CSA,Certified Organic,92026,-117.119996,33.195886,"{""x"": -117.11999569885434, ""y"": 33.19588635265..."


In [20]:
%%time
# Convert back from a SEDF into a feature layer, and publishing on AGOL
my_new_featurelayer = ca_farms_sgdf.spatial.to_featurelayer(title="San Diego County Farms", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['SanDiegoCountyFarms'])

CPU times: user 150 ms, sys: 22.3 ms, total: 172 ms
Wall time: 10.2 s
